In [ ]:
!pip install cv2
!pip install qrcode[pil] geopy twilio
!pip install qrcode

In [ ]:

import cv2
import qrcode
from geopy.geocoders import Nominatim
from twilio.rest import Client
from IPython.display import display
from google.colab import files
import numpy as np

# Function to get the live location link using geopy
def get_live_location_link():
    geolocator = Nominatim(user_agent="location_sender")
    location = geolocator.geocode("Your location")  # Replace with your actual location
    latitude = 18.605972
    longitude = 73.875045
    if location:
        # Construct a link to display the live location on a map
        link = f"https://www.google.com/maps?q={latitude},{longitude}"
        return link
    else:
        print("Error: Unable to retrieve live location.")
        return None

# Function to send SMS using Twilio
def send_sms(account_sid, auth_token, from_number, to_numbers, message):
    client = Client(account_sid, auth_token)

    for to_number in to_numbers:
        message = client.messages.create(
            body=message,
            from_=from_number,
            to=to_number
        )
        print(f"SMS sent to {to_number}, Message SID: {message.sid}")

# Function to read uploaded QR code image and initiate SMS sending
def process_uploaded_qr_code():
    # Prompt user to upload a file
    print("Please upload the QR code file.")
    uploaded_file = files.upload()

    # Check if a file was uploaded
    if len(uploaded_file) > 0:
        # Get the file name
        file_name = list(uploaded_file.keys())[0]

        # Read QR code data from the file
        with open(file_name, 'rb') as file:
            qr_code_data = file.read().decode('utf-8', errors='replace')

        decoded_info = cv2.QRCodeDetector().detectAndDecode(cv2.imread(file_name))[0]


        decoded_list=[]
        # Check if QR code was detected
        if decoded_info:
            # Split the concatenated information into individual elements
            decoded_list = [number for number in decoded_info.split(',') if number.strip()]  # Remove empty strings

        else:
            print("No QR Code detected.")

        # Access the decoded information as a list
        phone_numbers = decoded_list
        print(decoded_list)


        # Get live location link
        location_link = get_live_location_link()
        if location_link:
            # Construct the message with live location link
            message = f"Your Truck has meet with the accident at" +f"Live Location: {location_link}"

            # Twilio account credentials
            account_sid = 'ACa507a4c611b3b2ddaac029e3a6f031ae'
            auth_token = 'ea532b05151fe8ec5945491c8aa2a974'
            from_number = '+18287617395'

            # Send SMS to the specified phone numbers
            send_sms(account_sid, auth_token, from_number, phone_numbers, message)

# Run the process for uploaded QR code
process_uploaded_qr_code()
